In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("training_data_customers.csv")
final_df = df.drop(columns="Unnamed: 1")

In [2]:
from sklearn.ensemble import RandomForestRegressor

## Random Forest
# Best parameters found
best_params = {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}

# Create and train the Random Forest model with the best parameters
rf_model = RandomForestRegressor(random_state=42, **best_params)  # **best_params unpacks the dictionary

In [3]:
#Update Customer Data
def update_customer_data(new_customer_counts):
    """Updates customer data for shifts 1, 2, and 3 with rounded values.

    Args:
        new_customer_counts: A NumPy array or a list/tuple that can be converted
                             to a NumPy array of customer counts.

    Returns:
        The updated DataFrame.
    """

    global final_df

    new_customer_counts = np.array(new_customer_counts) # Ensure it's a NumPy array
    if len(new_customer_counts) != 3:
        raise ValueError("new_customer_counts must be a list or tuple of length 3.")


    rounded_counts = np.round(new_customer_counts).astype(int) # Round and convert to integers

    last_group = final_df['Group'].max()
    if pd.isna(last_group):
        new_group = 1
    else:
        new_group = last_group + 1

    new_rows = []
    for i, customers in enumerate(rounded_counts):
        new_rows.append({'Group': new_group, 'Shift': i + 1, 'Customers': customers})

    final_df = pd.concat([final_df, pd.DataFrame(new_rows)], ignore_index=True)
    return final_df

In [4]:
#Retraining the model
def retraining(final_df):
    """Retrains the model with the updated DataFrame.

    Args:
        final_df: The updated pandas DataFrame.

    Returns:
        The retrained model (it's generally better to return the model itself).
    """
    X = final_df[['Shift']]
    y = final_df['Customers']

    rf_model.fit(X, y)  # Retrain the model

    return rf_model

In [5]:
# Retrain the model:
retrained_model = retraining(final_df)  # Store the retrained model

#Now you can use predict then retrained the model:
customer_data = np.array([[1], [2], [3]])
new_predictions = retrained_model.predict(customer_data)
final_df = update_customer_data(new_predictions)

C:\Users\yuliastu\Music\Feni\Uni\Master Dokumen\Intelligent Agent\mesa-restaurant-agents\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [8]:
import mesa
print(mesa.__version__)

3.1.0


In [9]:
import mesa
import random 

# Data (should be accessible to the model)
shifts = [1, 2, 3]
demands = {1: 40, 2: 45, 3: 60}
capacity = 20

In [10]:
class Waiter(mesa.Agent):
    def __init__(self, model, is_fulltime, name):
        super().__init__(model)  # Correct initialization without unique_id
        self.is_fulltime = is_fulltime
        self.name = name
        self.shifts_worked = 0
        self.available = True

    def work_shift(self):
        self.shifts_worked += 1
        if self.is_fulltime:
            self.available = self.shifts_worked < 2  # Full-time waiters can work up to 2 shifts
        else:
            self.available = self.shifts_worked < 1  # Part-time waiters can work only 1 shift

    def reset_availability(self):
        self.available = True  # Always reset availability to True at the start of a new day
        self.shifts_worked = 0  # Reset shifts worked for the new day


#Solving Like this is not allowed

In [11]:
import math

class RestaurantModel(mesa.Model):
    def __init__(self, seed=None):
        super().__init__(seed=seed)  # Proper model initialization
        
        self.shifts = [1, 2, 3]
        self.demands = {1: 40, 2: 45, 3: 60}  # Demand per shift
        self.capacity = 20  # Each waiter serves 20 customers

        self.fulltime_names = ["Ana", "Bob", "Alice"]
        self.parttime_names = ["Laura", "Bill", "Adi", "Jon"]
        
        # Create waiters with unique IDs
        for name in self.fulltime_names:
            waiter = Waiter(model=self, is_fulltime=True, name=name) #Agent Waiter Initialization
            self.agents.add(waiter)
        for name in self.parttime_names:
            waiter = Waiter(model=self, is_fulltime=False, name=name)
            self.agents.add(waiter)
    
    def step(self):
        print("\n--- New Day ---")        
        # Reset availability for all waiters
        for waiter in self.agents:
            waiter.reset_availability()
            #print(f"Waiter {waiter.unique_id} ({waiter.name}): Available = {waiter.available}, Shifts Worked = {waiter.shifts_worked}")
            
            # Assign waiters to shifts based on demand
            assignments = {shift: [] for shift in self.shifts}
            fulltime_waiters_next_shift = []
            
        for shift in self.shifts:
            required_waiters = max(2, math.ceil(self.demands[shift] / self.capacity))  # Ensure at least 2 waiters per shift
            #print(f"Shift {shift}: Demand = {self.demands[shift]}, Required Waiters = {required_waiters}")
            
            available_waiters = [w for w in self.agents if w.available]
            #print(f"Available waiters: {[f'{w.unique_id} ({w.name})' for w in available_waiters]}")
            
            # Ensure full-time waiters can work 2 shifts consecutively
            fulltime_waiters = [w for w in available_waiters if w.is_fulltime and w.shifts_worked < 2]
            parttime_waiters = [w for w in available_waiters if not w.is_fulltime and w.shifts_worked < 1]
            #print(f"Full-time waiters: {[f'{w.unique_id} ({w.name})' for w in fulltime_waiters]}")
            #print(f"Part-time waiters: {[f'{w.unique_id} ({w.name})' for w in parttime_waiters]}")
            
            assigned_waiters = []
            
            # Assign full-time waiters from the previous shift if available
            if fulltime_waiters_next_shift:
                assigned_waiters = fulltime_waiters_next_shift
                fulltime_waiters_next_shift = []
                #print(f"Assigned full-time waiters from previous shift: {[f'{w.unique_id} ({w.name})' for w in assigned_waiters]}")
            
            # Assign additional waiters if needed
            if len(assigned_waiters) < required_waiters:
                remaining_waiters_needed = required_waiters - len(assigned_waiters)
                if len(fulltime_waiters) >= remaining_waiters_needed:
                    assigned_waiters += random.sample(fulltime_waiters, remaining_waiters_needed)
                else:
                    assigned_waiters += fulltime_waiters
                    remaining_waiters_needed -= len(fulltime_waiters)
                    if remaining_waiters_needed > 0:
                        assigned_waiters += random.sample(parttime_waiters, min(len(parttime_waiters), remaining_waiters_needed))
                #print(f"Assigned additional waiters: {[f'{w.unique_id} ({w.name})' for w in assigned_waiters]}")
            
            # Assign waiters to the shift and update their status
            for waiter in assigned_waiters:
                waiter.work_shift()
                assignments[shift].append(f"{waiter.unique_id} ({waiter.name})")
                if waiter.is_fulltime and waiter.shifts_worked < 2:
                    fulltime_waiters_next_shift.append(waiter)
            #print(f"Assignments after initial assignment: {assignments[shift]}")
        
            # Check if the demand is still not met and assign more waiters if needed
            if len(assignments[shift]) < required_waiters:
                remaining_waiters_needed = required_waiters - len(assignments[shift])
                additional_waiters = [w for w in available_waiters if w not in assigned_waiters]
                if len(additional_waiters) >= remaining_waiters_needed:
                    assigned_waiters += random.sample(additional_waiters, remaining_waiters_needed)
                else:
                    assigned_waiters += additional_waiters
                #print(f"Assigned additional waiters to meet demand: {[f'{w.unique_id} ({w.name})' for w in assigned_waiters]}")
                
                for waiter in assigned_waiters:
                    if f"{waiter.unique_id} ({waiter.name})" not in assignments[shift]:
                        waiter.work_shift()
                        assignments[shift].append(f"{waiter.unique_id} ({waiter.name})")
                        if waiter.is_fulltime and waiter.shifts_worked < 2:
                            fulltime_waiters_next_shift.append(waiter)
            #print(f"Final assignments for shift {shift}: {assignments[shift]}")
        
        # Print results for the day
        for shift, workers in assignments.items():
            print(f"Shift {shift}: {', '.join(workers) if workers else 'No waiters assigned'}")

# Run the model for multiple days
model = RestaurantModel(70)  # Seed ensures reproducibility
for _ in range(5):  # Simulate 5 days
    model.step()



--- New Day ---
Shift 1: 2 (Bob), 3 (Alice)
Shift 2: 2 (Bob), 3 (Alice), 1 (Ana)
Shift 3: 1 (Ana), 1 (Ana), 4 (Laura)

--- New Day ---
Shift 1: 2 (Bob), 1 (Ana)
Shift 2: 2 (Bob), 1 (Ana), 1 (Ana)
Shift 3: 3 (Alice), 6 (Adi), 7 (Jon)

--- New Day ---
Shift 1: 3 (Alice), 1 (Ana)
Shift 2: 3 (Alice), 1 (Ana), 2 (Bob)
Shift 3: 2 (Bob), 2 (Bob), 4 (Laura)

--- New Day ---
Shift 1: 3 (Alice), 1 (Ana)
Shift 2: 3 (Alice), 1 (Ana), 1 (Ana)
Shift 3: 2 (Bob), 5 (Bill), 7 (Jon)

--- New Day ---
Shift 1: 3 (Alice), 2 (Bob)
Shift 2: 3 (Alice), 2 (Bob), 3 (Alice)
Shift 3: 1 (Ana), 6 (Adi), 4 (Laura)


## USE LP to Solve Optimization

In [107]:
#Waiter Type
waiter_types = ["Fulltime", "Parttime"]
waite_total = {"Fulltime": 5, "Parttime": 4}
# Waiter Name
waiter_name = {
    "Fulltime": ["Ana", "Bob", "Alice", "Putri", "Lala"],
    "Parttime": ["Laura", "Bill", "Adi", "Jon"]
}

shifts = [1, 2, 3]
customer_demands = {1: 60, 2: 43, 3: 45}
capacity_waiter = 20

In [103]:
import pyoptinterface as poi
from pyoptinterface import highs

In [108]:
#optimization model with free solver HiGHS
model = highs.Model()

In [109]:
#Decision Variable
waiters = model.add_variables(waiter_types, domain=poi.VariableDomain.Integer, lb=0)

- Full-time waiters can work at most 2 shifts per day, without the requirement for continuous shifts.
- Part-time waiters can work at most 1 shift per day.
- The total capacity in each shift meets or exceeds customer demands.
- Each shift has at least 2 waiters.
- Only specific waiters can work in each shift based on the waiters_per_shift list.
- Group A und Group B has Relationship

Objective minimize number of waiters. Only allocate what i needed based on constraints

In [110]:
# Define variables for each waiter in each shift
waiter_vars = {}
for waiter_type in waiter_types:
    for waiter in waiter_name[waiter_type]:
        for shift in shifts:
            var_name = f"{waiter}_{shift}"
            waiter_vars[var_name] = model.add_variable(lb=0, ub=1, domain=poi.VariableDomain.Integer, name=var_name)
            
# Constraint: Each full-time waiter can work at most 2 shifts per day
for waiter in waiter_name["Fulltime"]:
    model.add_linear_constraint(
        poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for shift in shifts),
        poi.Leq,
        2,
        name=f"{waiter}_fulltime_max_two_shifts"
    )


# Constraint: Each part-time waiter can work at most 1 shift per day
for waiter in waiter_name["Parttime"]:
    model.add_linear_constraint(
        poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for shift in shifts),
        poi.Leq,
        1,
        name=f"{waiter}_parttime_max_one_shift"
    )

# Constraint: The total capacity in each shift must meet or exceed customer demands
for shift in shifts:
    model.add_linear_constraint(
        poi.quicksum(waiter_vars[f"{waiter}_{shift}"] * capacity_waiter for waiter_type in waiter_types for waiter in waiter_name[waiter_type]),
        poi.Geq,
        customer_demands[shift],
        name=f"shift_{shift}_demand"
    )

# Constraint: Each shift must have at least 2 waiters
for shift in shifts:
    model.add_linear_constraint(
        poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for waiter_type in waiter_types for waiter in waiter_name[waiter_type]),
        poi.Geq,
        2,
        name=f"shift_{shift}_min_two_waiters"
    )


In [91]:
# List of waiters that can work in each shift
waiters_per_shift = {
    1: ["Ana", "Bob", "Alice", "Putri"],
    2: ["Ana", "Bob", "Alice",  "Laura", "Bill", "Adi", "Jon"],
    3: ["Ana", "Bob", "Alice", "Putri", "Lala", "Laura", "Bill", "Adi", "Jon"]
}

# Additional constraint: Only specific waiters can work in each shift
for shift in shifts:
    for waiter_type in waiter_types:
        for waiter in waiter_name[waiter_type]:
            if waiter not in waiters_per_shift[shift]:
                model.add_linear_constraint(
                    waiter_vars[f"{waiter}_{shift}"],
                    poi.Eq,
                    0,
                    name=f"{waiter}_not_in_shift_{shift}"
                )

In [111]:
# Objective: Minimize the total number of waiters assigned
model.set_objective(
    poi.quicksum(waiter_vars[var] for var in waiter_vars),
    poi.ObjectiveSense.Minimize
)

model.set_model_attribute(poi.ModelAttribute.Silent, False)
model.optimize()

print(model.get_model_attribute(poi.ModelAttribute.TerminationStatus))
# TerminationStatusCode.OPTIMAL

# Get the values of the variables and print the schedule
schedule = {shift: [] for shift in shifts}
for var in waiter_vars:
    if model.get_value(waiter_vars[var]) > 0.5:  # If the waiter is assigned to the shift
        waiter, shift = var.rsplit('_', 1)
        schedule[int(shift)].append(waiter)

for shift in shifts:
    print(f"Shift {shift}: {', '.join(schedule[shift])}")

TerminationStatusCode.OPTIMAL
Shift 1: Ana, Bob, Lala
Shift 2: Alice, Putri, Adi
Shift 3: Putri, Lala, Laura


When the demand is high and there aren't enough waiters to meet all the constraints, there are a few strategies you can consider to handle the situation:

1. Relax Constraints
Adjust Shift Requirements: Allow part-time waiters to work more than one shift per day or allow full-time waiters to work more than two shifts.

2. Prioritize Constraints
Relax Less Critical Constraints: Identify which constraints are less critical and relax them to find a feasible solution.
Prioritize Critical Shifts: Ensure that the most critical shifts are fully staffed, even if it means understaffing less critical shifts.

3. Optimize Scheduling
Flexible Scheduling: Implement more flexible scheduling to better match waiter availability with demand.
Shift Swapping: Allow waiters to swap shifts to better accommodate their availability and preferences.

In [121]:
#Waiter Type
waiter_types = ["Fulltime", "Parttime"]
waite_total = {"Fulltime": 5, "Parttime": 4}
# Waiter Name
waiter_name = {
    "Fulltime": ["Ana", "Bob", "Alice", "Putri", "Lala"],
    "Parttime": ["Laura", "Bill", "Adi", "Jon"]
}

shifts = [1, 2, 3]
customer_demands = {1: 60, 2: 43, 3: 45}
capacity_waiter = 20

def solve_scheduling_problem(relax_constraints=False):
    model = highs.Model()

    #Decision Variable
    waiters = model.add_variables(waiter_types, domain=poi.VariableDomain.Integer, lb=0)

    # Define variables for each waiter in each shift
    waiter_vars = {}
    for waiter_type in waiter_types:
        for waiter in waiter_name[waiter_type]:
            for shift in shifts:
                var_name = f"{waiter}_{shift}"
                waiter_vars[var_name] = model.add_variable(lb=0, ub=1, domain=poi.VariableDomain.Integer, name=var_name)

    # List of waiters that can work in each shift
    waiters_per_shift = {
        1: ["Ana", "Bob", "Alice"],
        2: ["Ana", "Bob", "Laura", "Bill", "Adi", "Jon"],
        3: ["Alice", "Putri", "Lala", "Laura", "Bill", "Adi", "Jon"]
    }

    # Define groups
    group_A = ["Ana", "Bob", "Alice"]
    group_B = ["Putri", "Lala", "Laura"]

    # Constraint: Each full-time waiter can work at most 2 shifts per day
    for waiter in waiter_name["Fulltime"]:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for shift in shifts),
            poi.Leq,
            2 if not relax_constraints else 3,  # Relax constraint if needed
            name=f"{waiter}_fulltime_max_two_shifts"
        )

    # Constraint: Each part-time waiter can work at most 1 shift per day
    for waiter in waiter_name["Parttime"]:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for shift in shifts),
            poi.Leq,
            1 if not relax_constraints else 2,  # Relax constraint if needed
            name=f"{waiter}_parttime_max_one_shift"
        )

    # Constraint: The total capacity in each shift must meet or exceed customer demands
    for shift in shifts:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] * capacity_waiter for waiter_type in waiter_types for waiter in waiter_name[waiter_type]),
            poi.Geq,
            customer_demands[shift],
            name=f"shift_{shift}_demand"
        )

    # Constraint: Each shift must have at least 2 waiters
    for shift in shifts:
        model.add_linear_constraint(
            poi.quicksum(waiter_vars[f"{waiter}_{shift}"] for waiter_type in waiter_types for waiter in waiter_name[waiter_type]),
            poi.Geq,
            2,
            name=f"shift_{shift}_min_two_waiters"
        )

    # Additional constraint: Only specific waiters can work in each shift
    for shift in shifts:
        for waiter_type in waiter_types:
            for waiter in waiter_name[waiter_type]:
                if waiter not in waiters_per_shift[shift]:
                    model.add_linear_constraint(
                        waiter_vars[f"{waiter}_{shift}"],
                        poi.Eq,
                        0,
                        name=f"{waiter}_not_in_shift_{shift}"
                    )

    # Additional constraint: People in Group A and Group B cannot work together in the same shift
    for shift in shifts:
        for waiter_A in group_A:
            for waiter_B in group_B:
                model.add_linear_constraint(
                    waiter_vars[f"{waiter_A}_{shift}"] + waiter_vars[f"{waiter_B}_{shift}"],
                    poi.Leq,
                    1,
                    name=f"group_A_B_not_together_shift_{shift}"
                )

    # Objective: Minimize the total number of waiters assigned
    model.set_objective(
        poi.quicksum(waiter_vars[var] for var in waiter_vars),
        poi.ObjectiveSense.Minimize
    )

    model.set_model_attribute(poi.ModelAttribute.Silent, False)
    model.optimize()

    return model

# First attempt with original constraints
model = solve_scheduling_problem(relax_constraints=False)
print(model.get_model_attribute(poi.ModelAttribute.TerminationStatus))
# TerminationStatusCode.OPTIMAL
termination_status = model.get_model_attribute(poi.ModelAttribute.TerminationStatus)
print(f"Initial attempt termination status: {termination_status}")
print(f"Type of termination status: {type(termination_status)}")


# Check if a solution is found
if model.get_model_attribute(poi.ModelAttribute.TerminationStatus) != "TerminationStatusCode.OPTIMAL":
    # If no solution found, relax constraints and try again
    model = solve_scheduling_problem(relax_constraints=True)

# Print the schedule if a solution is found
if termination_status == poi.TerminationStatusCode.OPTIMAL:
    schedule = {shift: [] for shift in shifts}
    for var in waiter_vars:
        if model.get_value(waiter_vars[var]) > 0.5:  # If the waiter is assigned to the shift
            waiter, shift = var.rsplit('_', 1)
            schedule[int(shift)].append(waiter)

    for shift in shifts:
        print(f"Shift {shift}: {', '.join(schedule[shift])}")
else:
    print("No feasible solution found, even with relaxed constraints.")

TerminationStatusCode.OPTIMAL
Initial attempt termination status: TerminationStatusCode.OPTIMAL
Type of termination status: <enum 'TerminationStatusCode'>
Shift 1: Ana, Bob, Alice
Shift 2: Ana, Bob, Jon
Shift 3: Putri, Lala, Laura
